In [1]:
import pandas as pd
import time

time_start=time.time()
answer_info=pd.read_csv('answer_info.txt',sep='\s+',names=['aid','qid','uid','atime','ainfo','ainfoc','fine',
                                                                 'recommend','table','picture','video','numword',
                                                                 'thumbsup','thumbsdown','comment','collect','thank',
                                                                 'report','nohelp','oppose'])
invite_info=pd.read_csv('invite_info.txt',sep='\s+',names=['qid','uid','time','target'])
invite_info_evaluate=pd.read_csv('invite_info_evaluate_1.txt',sep='\s+',names=['qid','uid','time'])
#ques_info=pd.read_csv('question_info.txt',sep='\s+',names=['qid','qtime','qtitle','qtitlec','qinfo','qinfoc','qtopic'])
member_info=pd.read_csv('member_info.txt',sep='\s+',names=['uid','sex','key_word','num_level','hot_level','regis_type','regis_platform',
                                                          'look_freq','a','b','c','d','e','A','B','C','D','E','salt','l_topic','topic_n'])

In [2]:
member_info.drop_duplicates(['uid'],inplace=True)
train=invite_info[['qid','uid','target']]
test=invite_info_evaluate[['qid','uid']]
train=pd.merge(train,member_info,on='uid',how='left')
test=pd.merge(test,member_info,on='uid',how='left')

In [3]:
member_info['topic_n'] = member_info['topic_n'].astype(str).map(lambda x:x.split(','))
member_info['qtopic_len'] = member_info['topic_n'].map(lambda x:len(x))

In [4]:
from tqdm import tqdm

def flatten_topic(df):    
    u = []
    a = []
    q=[]
    for i in tqdm(range(len(df['topic_n'].values))):
        u += [df['uid'].values[i]]*df['qtopic_len'].values[i]
        a += list(df['topic_n'].values[i])
        
    new_df = pd.DataFrame()
    new_df['uid'] = u
    new_df['topic_n'] = a
        
    return new_df

In [5]:
member_info_flatten=flatten_topic(member_info)

100%|██████████| 1931654/1931654 [00:30<00:00, 63689.64it/s]


In [6]:
from tqdm import tqdm
tqdm.pandas()
member_info_flatten['topic_n_']=member_info_flatten['topic_n'].progress_apply(lambda x:x.split(':'))


100%|██████████| 12515611/12515611 [00:42<00:00, 293870.13it/s]


In [7]:
topic_n_tn=pd.DataFrame(member_info_flatten['topic_n_'].tolist())

In [8]:
deal_train=pd.concat([member_info_flatten,topic_n_tn],axis=1)

In [9]:
len(deal_train)

12515611

In [10]:
deal_train=deal_train[['uid',1]]

In [11]:
deal_train=deal_train.fillna(0)

In [12]:
deal_train[1]=deal_train[1].astype(float)

In [13]:
deal_train=deal_train.groupby(['uid']).agg(['mean','sum','max','std'])

In [14]:
deal_train=deal_train.reset_index()

In [15]:
deal_train.columns=['uid','topic_mean','topic_sum','topic_max','topic_std']

In [16]:
deal_train

,uid,topic_mean,topic_sum,topic_max,topic_std
0,M1000000382,0.748476,3.742382,1.059280,0.191178
1,M1000000983,1.686756,16.867561,1.922446,0.167220
2,M1000003304,1.097486,10.974862,1.291573,0.095671
3,M1000008978,0.739084,0.739084,0.739084,NaN
4,M1000009571,1.229389,1.229389,1.229389,NaN
5,M1000009786,0.613796,6.137958,0.962483,0.165138
6,M1000010662,0.000000,0.000000,0.000000,NaN
7,M1000013570,0.000000,0.000000,0.000000,NaN
8,M1000019448,0.982559,9.825588,1.325508,0.253057
9,M1000019749,0.000000,0.000000,0.000000,NaN


In [17]:
deal_train.to_csv('feature_topicn_tongji.csv',index=False)

In [18]:
time_end=time.time()
print('totally cost',time_end-time_start)

totally cost 277.9026207923889
